# ----------------------------------------------------------------------------------------------------------
# AWS DATA INGESTION
# ----------------------------------------------------------------------------------------------------------

___

<a> <img src='img\architecture_v2.png' width="1000" /></a>
___

### Import libraries

In [2]:
import boto3
import io 
import pandas as pd
import json
import time
import datetime
import numpy as np
import s3fs
# from fastparquet import write
import awswrangler
from datetime import datetime
from datetime import timedelta


## 1. Excel / Flat Files Data Sources

___

<a> <img src='img\xl_ingest_pic_v2.png' width="1000" /></a>
___

In [5]:
#define bucket address
bucket_name = 'mst-lab-data'
object_key = 'data/rawtest2.xlsx'

# s3 = boto3.client('s3', aws_access_key_id=aws_id, aws_secret_access_key=aws_secret)
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket_name, Key=object_key)
data = obj['Body'].read()
df = pd.read_excel(io.BytesIO(data), encoding='utf-8')
df.head()

,date,site,prod
0,2019-01-01,x,200
1,2019-01-02,x,100


In [4]:
# define target bucket to load the data
target_bucket='abm-data-platform-s3-raw'
target_object='rsw/test.csv'
csv_buffer = io.StringIO()
df.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object(target_bucket,target_object ).put(Body=csv_buffer.getvalue());

In [4]:
s3_url = 's3://blueprint-group-landzone/subgroup_1/datacatalog1/bucket.parquet.gzip'
df.to_parquet(s3_url, compression='gzip')

In [ ]:
write('output/outfile2.parq', df,compression='GZIP')

In [7]:
bytes_to_write = df.to_csv(None,index=False ).encode()
fs = s3fs.S3FileSystem()
with fs.open('s3://blueprint-group-rawdata/subgroup_1-rawdata/robert_raw1_csv/raw.csv', 'wb') as f:
    f.write(bytes_to_write)

In [51]:
import boto3
print (boto3.__version__)

1.9.127


In [8]:
dateTimeObj = datetime.now()
dateTimeObj

datetime.datetime(2019, 8, 19, 11, 18, 11, 350466)

In [9]:
yr= str(dateTimeObj.year)
mo= str(dateTimeObj.month)
day= str(dateTimeObj.day)
hr= str(dateTimeObj.hour)
mn= str(dateTimeObj.minute)
sc= str(dateTimeObj.second)
up_filename=yr+mo+day+hr+mn+sc+'.csv'
up_filename

'2019819111811.csv'

In [171]:
df2=df
df2['timestamp']=dateTimeObj
df2['useropluad']=object_key

df2

,date,site,prod,timestamp,useropluad
0,2019-01-01,x,100,2019-08-12 15:13:29.411861,abm/uploadhere/raw1.xlsx


In [46]:
a=str(dateTimeObj)
a

'2019-08-13 12:35:18.057543'

In [45]:
print(dateTimeObj)

2019-08-13 12:35:18.057543


In [124]:
df

,date,site,prod
0,2019-01-01,x,100


In [29]:
datarawcheck.columns = range(df.shape[1])
datarawcheck
cols_skip=[]
# pd.isna(datarawcheck[2])
for i in range(len(datarawcheck.columns)):
		if (((pd.isna(datarawcheck[i])).nunique()) == 1 ) & ((((pd.isna(datarawcheck[i])).unique())[0])==True):
			cols_skip.append(i)
		else:
			break
cols_skip


[]

In [30]:
cols = [i for i in range(len(df.columns)) if i not in cols_skip]
# 	rows=len(rows_skip)
cols

[0, 1, 2]

In [32]:
rows_skip=[]
for i in range(len(datarawcheck.iloc[i])):
	if (((pd.isna(datarawcheck.iloc[i])).nunique()) == 1 ) & ((((pd.isna(datarawcheck.iloc[i])).unique())[0])==True):
		rows_skip.append(i)
	else:
		break
rows_skip

[]

In [33]:
rows=len(rows_skip)
rows

0

In [125]:
write('output/outfile2.parq', df,compression='GZIP')
# write('outfile2.parq', df, row_group_offsets=[0, 10000, 20000], compression='GZIP', file_scheme='hive')

In [99]:
#Parameters
database = "abm"
table_name = "data"
s3_path = "s3://lz-layer/abm/"

sql_query = "SELECT * FROM mj LIMIT 20" 
db_name = "test-db"
s3_output_bucket = "s3://lz-layer/abm/"

# a = pg.read_glue(sql_query,db_name,s3_output_bucket,region='ap-southeast-1',key=aws_id,secret=aws_secret)

In [16]:
df['Year']= pd.to_datetime(df['date']).dt.strftime("%Y")
df['Month']= pd.to_datetime(df['date']).dt.strftime("%m")
df['Month']=pd.to_numeric(df['Month'])
df['Day']= pd.to_datetime(df['date']).dt.strftime("%d")

df['Day']=pd.to_numeric(df['Day'])
df.head()

,date,site,land_clearing,top_soil_removal,dump_to_rehabilitation,over_burden,overburden_removal_plan,rain,rainfall,slippery,...,blc_stockpile,coal_barging_olc_to_bg,coal_barging_blc_to_bg,coal_barging,coal_barging_plan,coal_shipping_actual,coal_shipping_plan,Year,Month,Day
0,2017-01-01,MFA-MBH,0.49,0.0,0,3384.0,13059.556,16.0,42.8,1.0,...,5618.00,0.0,0.0,0.0,5053.368581,2217,5053.0,2017,1,1
1,2017-01-02,MFA-MBH,1.08,0.0,0,8214.0,13059.556,2.2,4,12.8,...,16118.00,0.0,0.0,0.0,5053.368581,3925,5053.0,2017,1,2
2,2017-01-03,MFA-MBH,0.33,0.0,0,6813.0,13059.556,6.3,44.2,0.0,...,21118.00,0.0,0.0,0.0,5053.368581,3347,5053.0,2017,1,3
3,2017-01-04,MFA-MBH,1.02,0.0,0,1752.0,13059.556,14.8,39,9.0,...,23877.56,0.0,0.0,0.0,5053.368581,823,5053.0,2017,1,4
4,2017-01-05,MFA-MBH,0.44,0.0,0,1896.0,13059.556,20.4,53.2,3.4,...,28911.96,0.0,0.0,0.0,5053.368581,2196,5053.0,2017,1,5


## 2. Sharepoint List Data Sources

___

<a> <img src='pics\sharepoint challenge.png' width="600" /></a>
___

>Cloud Watch by schedule will trigger the Lambda function.

>Lambda function will call API – Sharepoint to read targeted list table to be ingested into AWS Data Platform and to process some actions as below : 

>The successfully uploaded file will trigger Lambda functions to process some actions as below : 
>   1. Re-check the template structure.
 
>   2. Filter the data who is not in landing zone –S3 bucket-folder yet.
 
>   3. Convert the processed data into parquet format.

>All the processed raw data will be loaded / inserted into targeted landing_zone S3 bucket-folder.

## 3. SQL DB Data Sources

___

<a> <img src='pics\sqldb challenge.png' width="600" /></a>
___

>Add connection for each SQL db using JDBC into AWS Data Platform as data catalog in a landing zonee database  (in targeted landing zone  S3 bucket-folder).

>By Schedule, AWS Glue will read the table and generate the hash-key unique.

>The python codes will recheck and filter the hash-key unique which is not in the existing loaded table already.

>Convert the processed data into parquet format.

>All the processed raw data will be loaded / inserted into targeted landing_zone S3 bucket-folder.